<a href="https://colab.research.google.com/github/Bart-del/studies-machine-learning/blob/main/Zadanie_4_Bart%C5%82omiej_Wysocki.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>Zadanie 4. Regresja liniowa wielu zmiennych

Stwórz model regresji liniowej dla cen mieszkań w Poznaniu. Zestaw danych to 5000 ogłoszeń z serwisu gratka.pl.

Miarą błędu na danych testowych niech będzie średnia kwadratowa błędu (Root Mean Squared Error, RMSE) dla cen mieszkań.

Plik gratkapl.csv zawiera dane, na których należy zbudować model regresyjny. Pierwszy wiersz w pliku to nagłówek zawierający opisy kolumn:

    Id - numer porządkowy oferty
    Expected - cena mieszkania
    Rooms - liczba pokojów
    SqrMeters - metraż mieszkania w m2
    Floor - piętro
    Location - dzielnica lub adres w Poznaniu
    Description - fragment opisu mieszkania

In [1]:
from google.colab import files
uploaded = files.upload()



import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Saving gratkapl.csv to gratkapl.csv


<h1>Wczytanie danych 

In [103]:
import csv
reader = csv.reader(open('gratkapl.csv', encoding='utf-8'), delimiter=',')
data = pd.DataFrame(reader)
new_header = data.iloc[0]
data = data[1:]
data.columns = new_header

<h1>Podzielenie danych na zbiór uczący i testowy

In [104]:
del data["Location"]
del data["Description"]
del data["Id"]

data['Expected'] = pd.to_numeric(data['Expected'])
data['Rooms'] = pd.to_numeric(data['Rooms'])
data['Floor'] = pd.to_numeric(data['Floor'])
data['SqrMeters'] = pd.to_numeric(data['SqrMeters'])

trening_set = data[:4000]
test_set = data[4000:]

<h1>Usuwanie wartości odstających

In [106]:
from scipy import stats
              
trening_set = trening_set[(np.abs(stats.zscore(trening_set)) < 3).all(axis=1)]
test_set = test_set[(np.abs(stats.zscore(test_set)) < 3).all(axis=1)]

test_set

,Expected,Rooms,SqrMeters,Floor
4001,356637,4,63.01,3
4002,280462,2,46.90,1
4003,249000,1,42.00,1
4004,315198,3,53.88,1
4005,369000,2,50.00,2
...,...,...,...,...
4996,405000,3,77.97,3
4997,330000,3,64.00,2
4998,460000,4,110.00,1
4999,435000,3,64.00,2
